In [4]:
import os
import pandas as pd
import numpy as np

win_num = input("Enter the winning number: ")
aurum_odds = input("Enter the Aurum odds: ")

user = os.getlogin()
path = fr"C:\Users\{user}\Desktop\Bet Selection History.xlsx"
pd.set_option('display.max_columns', 47)
df = pd.read_excel(path)

df.drop(['Game', 'Unnamed: 2', 'Table', 'Unnamed: 5', 'Unnamed: 13', 'Unnamed: 16', 'Unnamed: 22', 'Unnamed: 27', 'Unnamed: 36', 'ParentTable', 'Unnamed: 39', 'Unnamed: 40', 'Dealer', 'Unnamed: 42', 'Unnamed: 43', 'Step Id', 'Is Test', 'Group Id'], axis = 1, inplace = True)

df.columns = ['ID', 'Game', 'Table', 'Round ID', 'Client AGP ID', 'Client NickName', 'Bet EUR', 'Profit EUR', 'Selection ID', 'Selection', 'SelectionDetail ID', 'SelectionDetail', 'Selection Values', 'Odds', 'Bet Date', 'SelectionWinState ID', 'SelectionWinState', 'Bet Amount', 'Win Amount', 'Partner ID', 'Partner', 'Profit', 'Client SMT ID', 'Client External ID', 'Currency', 'Bet Id', 'Win EUR', 'Mode ID', 'Mode']

df.drop(index = 0, inplace = True)

odds = pd.read_csv(fr"C:\Users\{user}\Desktop\Roulette Odds.csv")

win_num_properties = odds[(odds['Coverted Selection Value Code'] == win_num) & (odds['Selection Code'] == 28)]

win_color = str(win_num_properties['Colour'].iloc[0]).rstrip('0').rstrip('.') if not win_num_properties['Colour'].empty else ''
win_even_odd = str(win_num_properties[r'Even/Odd'].iloc[0]).rstrip('0').rstrip('.') if not win_num_properties[r'Even/Odd'].empty else ''
win_high_low = str(win_num_properties[r'High/Low'].iloc[0]).rstrip('0').rstrip('.') if not win_num_properties[r'High/Low'].empty else ''
win_column = str(win_num_properties['Column'].iloc[0]).rstrip('0').rstrip('.') if not win_num_properties['Column'].empty else ''
win_dozen = str(win_num_properties['Dozen'].iloc[0]).rstrip('0').rstrip('.') if not win_num_properties['Dozen'].empty else ''

df['Search Selection Values'] = "," + df['Selection Values'] + ","

df['SelectionDetail ID'] = df['SelectionDetail ID'].astype(str)
df['Selection ID'] = df['Selection ID'].astype(str)
df['Search Selection Values'] = df['Search Selection Values'].astype(str)

df['Correct Result'] = np.where(
    ((df['SelectionDetail ID']).isin([win_color, win_even_odd,  win_high_low,  win_column, win_dozen])) |
    ((df['Selection ID'] == '28') & (df['Selection Values'] == win_num)) |
    ((df['Selection ID'] == '37') & (aurum_odds == '0') & (df['Selection Values'] == win_num)) |
    ((df['Selection ID'].isin(['29', '30', '31', '32']) & df['Search Selection Values'].str.contains(',' + win_num + ','))),
    df['Bet Amount'] * df['Odds'],
    np.where(
        ((df['Selection ID'] == '37') & (aurum_odds != '0') & (df['Selection Values'] == win_num)),
    df['Bet Amount'] * int(aurum_odds),
    0
    )
)

df['Correction Amount'] = df['Correct Result'] - df['Win Amount'] 

df_grouped = df.groupby(['Partner', 'Client AGP ID','Bet Date','Bet Id', 'Game', 'Table', 'Round ID', 'Currency'])[['Bet Amount', 'Bet EUR', 'Win Amount', 'Correct Result', 'Correction Amount']].sum().reset_index()

df_result = df_grouped[df_grouped['Correction Amount'] != 0].sort_values(by = ['Partner', 'Client AGP ID'])

df_result.to_excel(fr"C:\Users\{user}\Desktop\Round_Recalculated.xlsx", index = False)

Enter the winning number:  0
Enter the Aurum odds:  0
